In [ ]:
#packages
!pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import shutil
import time
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import requests
!pip install pandas_datareader
import pandas_datareader.data as web
from pandas.api.types import CategoricalDtype
import os
from openpyxl import load_workbook
from collections import defaultdict
from pathlib import Path
#Ticker from excel
arquivo = input('Nome do arquivo com os códigos na primeira coluna: ')
excel = pd.read_excel (arquivo+'.xlsx', header=None)
lstcvm = excel.iloc[:, 0].astype(str)
datas = input('Datas separadas por vírgula (Deixe vazio para todas): ')
#Manter dados brutos
manterraw = input('Digite "SIM" para manter arquivos brutos: ')
manterraw = manterraw.lower()
if manterraw == "sim":
    print("Dados brutos serão salvos")
else:
    print("Dados brutos serão excluídos")
#Inicio pelo código CVM
for i in lstcvm:
    driver = webdriver.Firefox()
    url="https://bvmf.bmfbovespa.com.br/cias-listadas/empresas-listadas/ResumoDemonstrativosFinanceiros.aspx?codigoCvm="+i+"&idioma=pt-br"
    setorurl="https://bvmf.bmfbovespa.com.br/cias-listadas/empresas-listadas/ResumoEmpresaPrincipal.aspx?codigoCvm="+i+"&idioma=pt-br"
    driver.get(url)
#Definição do nome da empresa
    try:
        ticker = driver.find_element(By.CSS_SELECTOR, "#ctl00_contentPlaceHolderConteudo_MenuEmpresasListadas1_lblNomeEmpresa").text
        tickerid = ticker.replace("S.A.","").replace(" ","_")
    except:
        ticker = "EMPRESA_NÃO_IDENTIFICADA"
#Verificação da validade do código CVM
    try:
        cvmdatas = driver.find_element(By.CSS_SELECTOR, "#ctl00_contentPlaceHolderConteudo_cmbAno").text
        cvmlistdatas = list(map(str,cvmdatas.split('\n')))
        cvmlistdatas = cvmlistdatas[1:]
    except:
        pass
    if cvmlistdatas:
        print("Código CVM válido: "+i+" ("+ticker+").")
#Coleta de dados de setor
        driver.get(setorurl)
        try:
            WebDriverWait(driver, 5).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"#ctl00_contentPlaceHolderConteudo_iframeCarregadorPaginaExterna")))
            setor = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/ul/li[1]/div/table/tbody/tr[5]/td[2]").get_attribute("innerHTML")
            setor = setor.replace(" / ","/").replace(".","").replace(" ","_")
            setor = setor.upper()
            setor
            print(setor)
        except:
            setor = "NÃO_IDENTIFICADO"
            print("Setor não identificado")
        driver.get(url)
        time.sleep(5)
#Listagem de datas inseridas pelo usuário
        if datas == "":
            datasb = driver.find_element(By.CSS_SELECTOR, "#ctl00_contentPlaceHolderConteudo_cmbAno").text
            listadatas = list(map(str,datasb.split('\n')))
            listadatasb = listadatas[1:]
            listadatasb.sort()
            listadatasc = ','.join(map(str, listadatasb))
            print("Datas: "+listadatasc+".")
#Listagem de datas total
        else:
            listadatasb = list(map(str,datas.split(',')))
            listadatasb.sort()
            listadatasc = ','.join(map(str, listadatasb))
            print("Datas: "+listadatasc+".")
#Seleção da data na página principal
        for z in listadatasb:
            try:
                Select(WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#ctl00_contentPlaceHolderConteudo_cmbAno")))).select_by_visible_text(z)
                if driver.find_elements(By.CSS_SELECTOR, "#ctl00_contentPlaceHolderConteudo_rptDocumentosDFP_ctl00_lnkDocumento"):
                    dfp = driver.find_element(By.CSS_SELECTOR, "#ctl00_contentPlaceHolderConteudo_rptDocumentosDFP_ctl00_lnkDocumento")
                    webdriver.ActionChains(driver).click(dfp).perform()
                    time.sleep(10)
                    tabs = driver.window_handles
                    driver.switch_to.window(tabs[1])
                    currenturl = driver.current_url
#Verificação de modelo do formulário BVMF
##Caso seja modelo antigo
                    if "bmfbovespa" in currenturl:
                        print("Formulário de "+ticker+" em "+z+" no modelo BVMF antigo.")
###Coleta de dados de balanço patrimonial ativo antigo
                        driver.get('https://www2.bmfbovespa.com.br/dxw/FormDetalheDXWBalanco.asp?TipoInfo=T&Tipo=01 - Ativo')
                        time.sleep(5)
                        try:
                            driver.find_elements(By.XPATH, "//*[contains(text(), 'Código')]")
                            balativoold = driver.find_element(By.CSS_SELECTOR, ".ScrollMaker > table:nth-child(1)").text
                            balativoold = balativoold.replace(".","").replace("(","-").replace(")","").replace("/","").replace(" ","\n")
                            balativoold = balativoold.splitlines()
                            balativoolddict = defaultdict(list)
                            key = None
                            reset_key = True
                            for item in balativoold:
                                if str(item).isdigit() or item.startswith('-'):
                                    if key not in balativoolddict:
                                        balativoolddict[key].append(item)
                                        reset_key = True
                                elif reset_key:
                                    key = item
                                    reset_key = False
                                else:
                                    key += f' {item}'
                            dict(balativoolddict)
                            dfbalativoold = pd.DataFrame(balativoolddict)
                            dfbalativoold = dfbalativoold.T
                            dfbalativoold = dfbalativoold.set_axis([z], axis=1, inplace=False)
                            dfbalativoold = dfbalativoold.iloc[1: , :]
                            dfbalativoold = dfbalativoold.rename_axis('Descrição')
###Criação do arquivo de balanço patrimonial ativo antigo
                            if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO_ANTIGO"):
                                os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO_ANTIGO")
                                dfbalativoold.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO_ANTIGO", tickerid+"_BALANCO_ATIVO_ANTIGO.xlsx"),sheet_name="BALANCO_ATIVO")
                            else:
                                excelbalativo = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO_ANTIGO/"+tickerid+"_BALANCO_ATIVO_ANTIGO.xlsx")
                                excelbalativo = excelbalativo.set_index("Descrição")
                                excelbalativo[z] = dfbalativoold[z]
                                excelbalativo.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO_ANTIGO/", tickerid+"_BALANCO_ATIVO_ANTIGO.xlsx"),sheet_name="BALANCO_ATIVO")
                        except:
                            print("Balanço Ativo de "+ticker+" em "+z+" sem dados.")
###Coleta de dados de balanço patrimonial passivo antigo
                        driver.get('https://www2.bmfbovespa.com.br/dxw/FormDetalheDXWBalanco.asp?TipoInfo=T&Tipo=02 - Passivo')
                        time.sleep(5)
                        try:
                            driver.find_elements(By.XPATH, "//*[contains(text(), 'Código')]")
                            balpassold = driver.find_element(By.CSS_SELECTOR, ".ScrollMaker > table:nth-child(1)").text
                            balpassold = balpassold.replace(".","").replace("(","-").replace(")","").replace("/","").replace(" ","\n")
                            balpassold = balpassold.splitlines()
                            balpassolddict = defaultdict(list)
                            key = None
                            reset_key = True
                            for item in balpassold:
                                if str(item).isdigit() or item.startswith('-'):
                                    if key not in balpassolddict:
                                        balpassolddict[key].append(item)
                                        reset_key = True
                                elif reset_key:
                                    key = item
                                    reset_key = False
                                else:
                                    key += f' {item}'
                            dict(balpassolddict)
                            dfbalpassold = pd.DataFrame(balpassolddict)
                            dfbalpassold = dfbalpassold.T
                            dfbalpassold = dfbalpassold.set_axis([z], axis=1, inplace=False)
                            dfbalpassold = dfbalpassold.iloc[1: , :]
                            dfbalpassold = dfbalpassold.rename_axis('Descrição')
###Criação do arquivo de balanço patrimonial passivo antigo
                            if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO_ANTIGO"):
                                os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO_ANTIGO")
                                dfbalpassold.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO_ANTIGO", tickerid+"_BALANCO_PASSIVO_ANTIGO.xlsx"),sheet_name="BALANCO_PASSIVO")
                            else:
                                excelbalpassivo = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO_ANTIGO/"+tickerid+"_BALANCO_PASSIVO_ANTIGO.xlsx")
                                excelbalpassivo = excelbalpassivo.set_index("Descrição")
                                excelbalpassivo[z] = dfbalpassold[z]
                                excelbalpassivo.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO_ANTIGO/", tickerid+"_BALANCO_PASSIVO_ANTIGO.xlsx"),sheet_name="BALANCO_PASSIVO")
                        except:
                            print("Balanço Passivo de "+ticker+" em "+z+" sem dados.")
###Coleta de dados de demonstração de resultados antigo
                        driver.get('https://www2.bmfbovespa.com.br/dxw/FormDetalheDXWDRE.asp?TipoInfo=C')
                        time.sleep(5)
                        try:
                            driver.find_elements(By.XPATH, "//*[contains(text(), 'Código')]")
                            demresold = driver.find_element(By.CSS_SELECTOR, ".ScrollMaker > table:nth-child(1)").text
                            demresold = demresold.replace(".","").replace("(","-").replace(")","").replace("/","").replace(" ","\n")
                            demresold = demresold.splitlines()
                            demresolddict = defaultdict(list)
                            key = None
                            reset_key = True
                            for item in demresold:
                                if str(item).isdigit() or item.startswith('-'):
                                    if key not in demresolddict:
                                        demresolddict[key].append(item)
                                        reset_key = True
                                elif reset_key:
                                    key = item
                                    reset_key = False
                                else:
                                    key += f' {item}'
                            dict(demresolddict)
                            dfdemresold = pd.DataFrame(demresolddict)
                            dfdemresold = dfdemresold.T
                            dfdemresold = dfdemresold.set_axis([z], axis=1, inplace=False)
                            dfdemresold = dfdemresold.iloc[1: , :]
                            dfdemresold = dfdemresold.rename_axis('Descrição')
###Criação do arquivo de demonstração de resultados antigo
                            if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ANTIGO"):
                                os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ANTIGO")
                                dfdemresold.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ANTIGO", tickerid+"_RESULTADOS_ANTIGO.xlsx"),sheet_name="RESULTADOS")
                            else:
                                exceldemres = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ANTIGO/"+tickerid+"_RESULTADOS_ANTIGO.xlsx")
                                exceldemres = exceldemres.set_index("Descrição")
                                exceldemres[z] = dfdemresold[z]
                                exceldemres.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ANTIGO/", tickerid+"_RESULTADOS_ANTIGO.xlsx"),sheet_name="RESULTADOS")
                        except:
                            print("Resultados de "+ticker+" em "+z+" sem dados.")
###Coleta de dados de fluxo de caixa antigo
                        driver.get('https://www2.bmfbovespa.com.br/dxw/FormDetalheDXWDFC.asp?TipoInfo=T&TipoDem=F')
                        time.sleep(5)
                        try:
                            driver.find_element(By.CSS_SELECTOR, ".ScrollMaker > table:nth-child(1)")
                            flxold = driver.find_element(By.CSS_SELECTOR, ".ScrollMaker > table:nth-child(1)").text
                            flxold = flxold.replace(".","").replace("(","-").replace(")","").replace("/","").replace(" ","\n")
                            flxold = flxold.splitlines()
                            flxolddict = defaultdict(list)
                            key = None
                            reset_key = True
                            for item in flxold:
                                if str(item).isdigit() or item.startswith('-'):
                                    if key not in flxolddict:
                                        flxolddict[key].append(item)
                                        reset_key = True
                                elif reset_key:
                                    key = item
                                    reset_key = False
                                else:
                                    key += f' {item}'
                            dict(flxolddict)
                            dfflxold = pd.DataFrame(flxolddict)
                            dfflxold = dfflxold.T
                            dfflxold = dfflxold.set_axis([z], axis=1, inplace=False)
                            dfflxold = dfflxold.iloc[1: , :]
                            dfflxold = dfflxold.rename_axis('Descrição')
###Criação do arquivo de fluxo de caixa antigo
                            if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"FLUXO_ANTIGO"):
                                os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"FLUXO_ANTIGO")
                                dfflxold.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"FLUXO_ANTIGO", tickerid+"_FLUXO_ANTIGO.xlsx"),sheet_name="FLUXO")
                            else:
                                excelflx = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"FLUXO_ANTIGO/"+tickerid+"_FLUXO_ANTIGO.xlsx")
                                excelflx = excelflx.set_index("Descrição")
                                excelflx[z] = dfflxold[z]
                                excelflx.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"FLUXO_ANTIGO/", tickerid+"_FLUXO_ANTIGO.xlsx"),sheet_name="FLUXO")
                        except:
                            print("Fluxo de Caixa de "+ticker+" em "+z+" sem dados.")
###Coleta de dados de valor adicionado antigo
                        driver.get('https://www2.bmfbovespa.com.br/dxw/FormDetalheDXWDFC.asp?TipoInfo=T&TipoDem=V')
                        time.sleep(5)
                        try:
                            driver.find_elements(By.XPATH, "//*[contains(text(), 'Código')]")
                            valadold = driver.find_element(By.CSS_SELECTOR, ".ScrollMaker > table:nth-child(1)").text
                            valadold = valadold.replace(".","").replace("(","-").replace(")","").replace("/","").replace(" ","\n")
                            valadold = valadold.splitlines()
                            valadolddict = defaultdict(list)
                            key = None
                            reset_key = True
                            for item in valadold:
                                if str(item).isdigit() or item.startswith('-'):
                                    if key not in valadolddict:
                                        valadolddict[key].append(item)
                                        reset_key = True
                                elif reset_key:
                                    key = item
                                    reset_key = False
                                else:
                                    key += f' {item}'
                            dict(valadolddict)
                            dfvaladold = pd.DataFrame(valadolddict)
                            dfvaladold = dfvaladold.T
                            dfvaladold = dfvaladold.set_axis([z], axis=1, inplace=False)
                            dfvaladold = dfvaladold.iloc[1: , :]
                            dfvaladold = dfvaladold.rename_axis('Descrição')
###Criação do arquivo de valor adicionado antigo
                            if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO_ANTIGO"):
                                os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO_ANTIGO")
                                dfvaladold.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO_ANTIGO", tickerid+"_VALOR_ADICIONADO_ANTIGO.xlsx"),sheet_name="VALOR_ADICIONADO_ANTIGO")
                            else:
                                excelvalad = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO_ANTIGO/"+tickerid+"_VALOR_ADICIONADO_ANTIGO.xlsx")
                                excelvalad = excelvalad.set_index("Descrição")
                                excelvalad[z] = dfvaladold[z]
                                excelvalad.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO_ANTIGO/", tickerid+"_VALOR_ADICIONADO_ANTIGO.xlsx"),sheet_name="VALOR_ADICIONADO_ANTIGO")
                        except:
                            print("Valor adicionado de "+ticker+" em "+z+" sem dados.")
#Retornar para a aba principal e trocar a data (reinicio do script)
                        driver.close()
                        driver.switch_to.window(tabs[0])
##Caso seja modelo novo
                    elif "blank" in currenturl:
                        print("Página não carregada para "+ticker+" em "+z+".")
                        driver.close()
                        driver.switch_to.window(tabs[0])
                    else:
                        print("Formulário de "+ticker+" em "+z+" no modelo BVMF novo.")
###Coleta de dados de balanço_patrimonial_ativo novo
                        Select(WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "select#cmbQuadro")))).select_by_visible_text("Balanço Patrimonial Ativo")
                        try:
                            WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe#iFrameFormulariosFilho")))
                            balativo  = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "table#ctl00_cphPopUp_tbDados"))).get_attribute("outerHTML")
                            balativo = balativo.replace(".","")
                            dfbalativo = pd.read_html(balativo)
                        except:
                            pass
###Coleta de dados de balanço_patrimonial_passivo novo
                        driver.get(currenturl)
                        Select(WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "select#cmbQuadro")))).select_by_visible_text("Balanço Patrimonial Passivo")
                        try:
                            WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe#iFrameFormulariosFilho")))
                            balpassivo  = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "table#ctl00_cphPopUp_tbDados"))).get_attribute("outerHTML")
                            balpassivo = balpassivo.replace(".","")    
                            dfbalpassivo = pd.read_html(balpassivo)
                        except:
                            pass
###Coleta de dados de demonstração_do_resultado novo
                        driver.get(currenturl)
                        Select(WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "select#cmbQuadro")))).select_by_visible_text("Demonstração do Resultado")
                        try:
                            WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe#iFrameFormulariosFilho")))
                            demres  = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "table#ctl00_cphPopUp_tbDados"))).get_attribute("outerHTML")
                            demres = demres.replace(".","")    
                            dfdemres = pd.read_html(demres)
                        except:
                            pass
###Coleta de dados de demonstração_do_resultado_abrangente novo
                        driver.get(currenturl)
                        Select(WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "select#cmbQuadro")))).select_by_visible_text("Demonstração do Resultado Abrangente")
                        try:
                            WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe#iFrameFormulariosFilho")))
                            demresa  = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "table#ctl00_cphPopUp_tbDados"))).get_attribute("outerHTML")
                            demresa = demresa.replace(".","")    
                            dfdemresa = pd.read_html(demresa)
                        except:
                            pass
###Coleta de dados de fluxo_de_caixa novo
                        driver.get(currenturl)
                        Select(WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "select#cmbQuadro")))).select_by_visible_text("Demonstração do Fluxo de Caixa")
                        try:
                            WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe#iFrameFormulariosFilho")))
                            flx  = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "table#ctl00_cphPopUp_tbDados"))).get_attribute("outerHTML")
                            flx = flx.replace(".","")    
                            dfflx = pd.read_html(flx)
                        except:
                            pass
###Coleta de dados de valor_adicionado novo
                        driver.get(currenturl)
                        Select(WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "select#cmbQuadro")))).select_by_visible_text("Demonstração de Valor Adicionado")
                        try:
                            WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe#iFrameFormulariosFilho")))
                            valad  = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "table#ctl00_cphPopUp_tbDados"))).get_attribute("outerHTML")
                            valad = valad.replace(".","")    
                            dfvalad = pd.read_html(valad)
                        except:
                            pass
###Fechando abas
                        driver.close()
                        driver.switch_to.window(tabs[0])
###Criação das dataframes novo
                        dfbalativob = pd.DataFrame(dfbalativo[0])
                        dfbalpassivob = pd.DataFrame(dfbalpassivo[0])
                        dfdemresb = pd.DataFrame(dfdemres[0])
                        dfdemresab = pd.DataFrame(dfdemresa[0])
                        dfflxb = pd.DataFrame(dfflx[0])
                        dfvaladb = pd.DataFrame(dfvalad[0])
###SAVE_RAW da dataframe de Balanço ativo novo
                        dfbalativob.columns = dfbalativob.iloc[0]
                        dfbalativob = dfbalativob.drop(0)
                        dfbalativob = dfbalativob.reset_index(drop=True)
                        dfbalativoraw = dfbalativob.set_index("Descrição")
                        dfbalativoraw = dfbalativoraw[dfbalativoraw.columns[1]]
                        dfbalativoraw = pd.DataFrame(dfbalativoraw)
                        dfbalativoraw.columns = [z]
                        dfbalativoraw = dfbalativoraw.astype(float)
                        dfbalativoraw = dfbalativoraw.fillna(0)
###Organização da dataframe de Balanço ativo novo
                        dfbalativob = dfbalativob.drop_duplicates(subset=['Descrição'])
                        dfbalativob = dfbalativob.set_index("Descrição")
                        dfbalativob = dfbalativob[dfbalativob.columns[1]]
                        dfbalativob = pd.DataFrame(dfbalativob)
                        dfbalativob.columns = [z]
                        dfbalativob = dfbalativob.astype(float)
                        dfbalativob = dfbalativob.fillna(0)
###SAVE_RAW da dataframe de Balanço passivo novo
                        dfbalpassivob.columns = dfbalpassivob.iloc[0]
                        dfbalpassivob = dfbalpassivob.drop(0)
                        dfbalpassivob = dfbalpassivob.reset_index(drop=True)
                        dfbalpassivoraw = dfbalpassivob.set_index("Descrição")
                        dfbalpassivoraw = dfbalpassivoraw[dfbalpassivoraw.columns[1]]
                        dfbalpassivoraw = pd.DataFrame(dfbalpassivoraw)
                        dfbalpassivoraw.columns = [z]
                        dfbalpassivoraw = dfbalpassivoraw.astype(float)
                        dfbalpassivoraw = dfbalpassivoraw.fillna(0)
###Organização da dataframe de Balanço passivo novo
                        dfbalpassivob = dfbalpassivob.drop_duplicates(subset=['Descrição'])
                        dfbalpassivob = dfbalpassivob.set_index("Descrição")
                        dfbalpassivob = dfbalpassivob[dfbalpassivob.columns[1]]
                        dfbalpassivob = pd.DataFrame(dfbalpassivob)
                        dfbalpassivob.columns = [z]
                        dfbalpassivob = dfbalpassivob.astype(float)
                        dfbalpassivob = dfbalpassivob.fillna(0)
###SAVE_RAW da dataframe de Demonstração de resultados novo
                        dfdemresb.columns = dfdemresb.iloc[0]
                        dfdemresb = dfdemresb.drop(0)
                        dfdemresb = dfdemresb.reset_index(drop=True)
                        dfdemresraw = dfdemresb.set_index("Descrição")
                        dfdemresraw = dfdemresraw[dfdemresraw.columns[1]]
                        dfdemresraw = pd.DataFrame(dfdemresraw)
                        dfdemresraw.columns = [z]
                        dfdemresraw = dfdemresraw.astype(float)
                        dfdemresraw = dfdemresraw.fillna(0)
###Organização da dataframe de Demonstração de resultados novo
                        dfdemresb = dfdemresb.drop_duplicates(subset=['Descrição'])
                        dfdemresb = dfdemresb.set_index("Descrição")
                        dfdemresb = dfdemresb[dfdemresb.columns[1]]
                        dfdemresb = pd.DataFrame(dfdemresb)
                        dfdemresb.columns = [z]
                        dfdemresb = dfdemresb.astype(float)
                        dfdemresb = dfdemresb.fillna(0)
###SAVE_RAW da dataframe de Demonstração de resultados abrangente novo
                        dfdemresab.columns = dfdemresab.iloc[0]
                        dfdemresab = dfdemresab.drop(0)
                        dfdemresab = dfdemresab.reset_index(drop=True)
                        dfdemresaraw = dfdemresab.set_index("Descrição")
                        dfdemresaraw = dfdemresaraw[dfdemresaraw.columns[1]]
                        dfdemresaraw = pd.DataFrame(dfdemresaraw)
                        dfdemresaraw.columns = [z]
                        dfdemresaraw = dfdemresaraw.astype(float)
                        dfdemresaraw = dfdemresaraw.fillna(0)
###Organização da dataframe de Demonstração de resultados abrangente novo
                        dfdemresab = dfdemresab.drop_duplicates(subset=['Descrição'])
                        dfdemresab = dfdemresab.set_index("Descrição")
                        dfdemresab = dfdemresab[dfdemresab.columns[1]]
                        dfdemresab = pd.DataFrame(dfdemresab)
                        dfdemresab.columns = [z]
                        dfdemresab = dfdemresab.astype(float)
                        dfdemresab = dfdemresab.fillna(0)
###SAVE_RAW da dataframe de Fluxo de caixa novo
                        dfflxb.columns = dfflxb.iloc[0]
                        dfflxb = dfflxb.drop(0)
                        dfflxb = dfflxb.reset_index(drop=True)
                        dfflxraw = dfflxb.set_index("Descrição")
                        dfflxraw = dfflxraw[dfflxraw.columns[1]]
                        dfflxraw = pd.DataFrame(dfflxraw)
                        dfflxraw.columns = [z]
                        dfflxraw = dfflxraw.astype(float)
                        dfflxraw = dfflxraw.fillna(0)
###Organização da dataframe de Fluxo de caixa novo
                        dfflxb = dfflxb.drop_duplicates(subset=['Descrição'])
                        dfflxb = dfflxb.set_index("Descrição")
                        dfflxb = dfflxb[dfflxb.columns[1]]
                        dfflxb = pd.DataFrame(dfflxb)
                        dfflxb.columns = [z]
                        dfflxb = dfflxb.astype(float)
                        dfflxb = dfflxb.fillna(0)
###Organização da dataframe de Valor adicionado novo
                        dfvaladb.columns = dfvaladb.iloc[0]
                        dfvaladb = dfvaladb.drop(0)
                        dfvaladb = dfvaladb.reset_index(drop=True)
                        dfvaladraw = dfvaladb.set_index("Descrição")
                        dfvaladraw = dfvaladraw[dfvaladraw.columns[1]]
                        dfvaladraw = pd.DataFrame(dfvaladraw)
                        dfvaladraw.columns = [z]
                        dfvaladraw = dfvaladraw.astype(float)
                        dfvaladraw = dfvaladraw.fillna(0)
###Organização da dataframe de Valor adicionado novo
                        dfvaladb = dfvaladb.drop_duplicates(subset=['Descrição'])
                        dfvaladb = dfvaladb.set_index("Descrição")
                        dfvaladb = dfvaladb[dfvaladb.columns[1]]
                        dfvaladb = pd.DataFrame(dfvaladb)
                        dfvaladb.columns = [z]
                        dfvaladb = dfvaladb.astype(float)
                        dfvaladb = dfvaladb.fillna(0)
###Criação do arquivo para Balanço ativo novo
                        if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO"):
                            os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO")
                            dfbalativob.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO", tickerid+"_BALANCO_ATIVO.xlsx"),sheet_name="BALANCO_ATIVO")
                        else:
                            excelbalativon = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO/"+tickerid+"_BALANCO_ATIVO.xlsx")
                            excelbalativon = excelbalativon.set_index("Descrição")
                            excelbalativon[z] = dfbalativob[z]
                            excelbalativon.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO", tickerid+"_BALANCO_ATIVO.xlsx"),sheet_name="BALANCO_ATIVO")
###Criação do arquivo para Balanço passivo novo
                        if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO"):
                            os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO")
                            dfbalpassivob.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO", tickerid+"_BALANCO_PASSIVO.xlsx"),sheet_name="BALANCO_PASSIVO")
                        else:
                            excelbalpassn = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO/"+tickerid+"_BALANCO_PASSIVO.xlsx")
                            excelbalpassn = excelbalpassn.set_index("Descrição")
                            excelbalpassn[z] = dfbalpassivob[z]
                            excelbalpassn.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO", tickerid+"_BALANCO_PASSIVO.xlsx"),sheet_name="BALANCO_PASSIVO")
###Criação do arquivo para Resultados novo
                        if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS"):
                            os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS")
                            dfdemresb.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS", tickerid+"_RESULTADOS.xlsx"),sheet_name="RESULTADOS")
                        else:
                            excelresn = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS/"+tickerid+"_RESULTADOS.xlsx")
                            excelresn = excelresn.set_index("Descrição")
                            excelresn[z] = dfdemresb[z]
                            excelresn.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS", tickerid+"_RESULTADOS.xlsx"),sheet_name="RESULTADOS")
###Criação do arquivo para Resultados Abrangentes novo
                        if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ABRANGENTES"):
                            os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ABRANGENTES")
                            dfdemresab.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ABRANGENTES", tickerid+"_RESULTADOS_ABRANGENTES.xlsx"),sheet_name="RESULTADOS_ABRANGENTES")
                        else:
                            excelresab = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ABRANGENTES/"+tickerid+"_RESULTADOS_ABRANGENTES.xlsx")
                            excelresab = excelresab.set_index("Descrição")
                            excelresab[z] = dfdemresab[z]
                            excelresab.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ABRANGENTES", tickerid+"_RESULTADOS_ABRANGENTES.xlsx"),sheet_name="RESULTADOS_ABRANGENTES")
###Criação do arquivo para Fluxo de caixa novo
                        if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"FLUXON"):
                            os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"FLUXON")
                            dfflxb.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"FLUXON", tickerid+"_FLUXON.xlsx"),sheet_name="FLUXON")
                        else:
                            excelflxn = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"FLUXON/"+tickerid+"_FLUXON.xlsx")
                            excelflxn = excelflxn.set_index("Descrição")
                            excelflxn[z] = dfflxb[z]
                            excelflxn.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"FLUXON", tickerid+"_FLUXON.xlsx"),sheet_name="FLUXON")
###Criação do arquivo para Valor adicionado novo
                        if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO"):
                            os.makedirs("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO")
                            dfvaladb.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO", tickerid+"_VALOR_ADICIONADO.xlsx"),sheet_name="VALOR_ADICIONADO")
                        else:
                            excelvaladn = pd.read_excel ("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO/"+tickerid+"_VALOR_ADICIONADO.xlsx")
                            excelvaladn = excelvaladn.set_index("Descrição")
                            excelvaladn[z] = dfvaladb[z]
                            excelvaladn.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO", tickerid+"_VALOR_ADICIONADO.xlsx"),sheet_name="VALOR_ADICIONADO")
###Criação arquivos raw
                        if not os.path.exists("BVMF/"+setor+"/"+tickerid+"/RAW"):
                            os.makedirs("BVMF/"+setor+"/"+tickerid+"/RAW")
                        try:
                            dfbalativoraw.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/RAW", tickerid+z+"_BALANCO_ATIVO.xlsx"),sheet_name="BALANCO_ATIVO")
                        except:
                            pass
                        try:
                            dfbalpassivoraw.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/RAW", tickerid+z+"_BALANCO_PASSIVO.xlsx"),sheet_name="BALANCO_PASSIVO")
                        except:
                            pass
                        try:
                            dfdemresraw.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/RAW", tickerid+z+"_RESULTADOS.xlsx"),sheet_name="RESULTADOS")
                        except:
                            pass
                        try:
                            dfdemresaraw.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/RAW", tickerid+z+"_RESULTADOS_ABRANGENTES.xlsx"),sheet_name="RESULTADOS_ABRANGENTES")
                        except:
                            pass
                        try:
                            dfflxraw.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/RAW", tickerid+z+"_FLUXON.xlsx"),sheet_name="FLUXON")
                        except:
                            pass
                        try:
                            dfvaladraw.to_excel(os.path.join("BVMF/"+setor+"/"+tickerid+"/RAW", tickerid+z+"_VALOR_ADICIONADO.xlsx"),sheet_name="VALOR_ADICIONADO")
                        except:
                            pass
#Fechamento do try de datas
            except:
                print(ticker+" não possui dados para "+z+".")
                try:
                    alert = driver.find_element(By.CSS_SELECTOR, "span.ui-button-text:nth-child(1)")
                    webdriver.ActionChains(driver).click(alert).perform()
                    driver.close()
                    driver.switch_to.window(tabs[0])
                except:
                    pass
        print(tickerid+" completo.")
        driver.quit()
        #Definindo dataframes:
        ##Balanço ativo antigo
        try:
            excelbalativo
            finalbalativo=excelbalativo
            finalbalativo = finalbalativo.fillna(0)
            finalbalativo = finalbalativo.T
        except:
            try:
                dfbalativoold
                finalbalativo=dfbalativoold
                finalbalativo = finalbalativo.fillna(0)
                finalbalativo = finalbalativo.T
            except:
                finalbalativo = {'Ativo Total': []}
                finalbalativo = pd.DataFrame(finalbalativo)
        ##Balanço passivo antigo
        try:
            excelbalpassivo
            finalbalpassivo=excelbalpassivo
            finalbalpassivo = finalbalpassivo.fillna(0)
            finalbalpassivo = finalbalpassivo.T
        except:
            try:
                dfbalpassold
                finalbalpassivo=dfbalpassold
                finalbalpassivo = finalbalpassivo.fillna(0)
                finalbalpassivo = finalbalpassivo.T
            except:
                finalbalpassivo = {'Passivo Total': [],'Patrimônio Líquido': []}
                finalbalpassivo = pd.DataFrame(finalbalpassivo)
        ##Resultados antigo
        try:
            exceldemres
            finaldemres=exceldemres
            finaldemres = finaldemres.fillna(0)
            finaldemres = finaldemres.T
        except:
            try:
                dfdemresold
                finaldemres=dfdemresold
                finaldemres = finaldemres.fillna(0)
                finaldemres = finaldemres.T
            except:
                finaldemres = {'EBITDA': []}
                finaldemres = pd.DataFrame(finaldemres)
        ##Fluxo antigo
        try:
            excelflx
            finalflx=excelflx
            finalflx = finalflx.fillna(0)
            finalflx = finalflx.T
        except:
            try:
                dfflxold
                finalflx=dfflxold
                finalflx = finalflx.fillna(0)
                finalflx = finalflx.T
            except:
                finalflx = {'Saldo Final de Caixa e Equivalentes': []}
                finalflx = pd.DataFrame(finalflx)
        ##Valor adicionado antigo
        try:
            excelvalad
            finalvalad=excelvalad
            finalvalad = finalvalad.fillna(0)
            finalvalad = finalvalad.T
        except:
            try:
                dfvaladold
                finalvalad=dfvaladold
                finalvalad = finalvalad.fillna(0)
                finalvalad = finalvalad.T
            except:
                finalvalad = {'': []}
                finalvalad = pd.DataFrame(finalvalad)
        ##Balanço ativo novo
        try:
            excelbalativon
            finalbalativon=excelbalativon
            finalbalativon = finalbalativon.fillna(0)
            finalbalativon = finalbalativon.T
        except:
            try:
                dfbalativob
                finalbalativon=dfbalativob
                finalbalativon = finalbalativon.fillna(0)
                finalbalativon = finalbalativon.T
            except:
                finalbalativon = {'Ativo Total': []}
                finalbalativon = pd.DataFrame(finalbalativon)
        ##Balanço passivo novo
        try:
            excelbalpassn
            finalbalpassn=excelbalpassn
            finalbalpassn = finalbalpassn.fillna(0)
            finalbalpassn = finalbalpassn.T
        except:
            try:
                dfbalpassivob
                finalbalpassn=dfbalpassivob
                finalbalpassn = finalbalpassn.fillna(0)
                finalbalpassn = finalbalpassn.T
            except:
                finalbalpassn = {'Passivo Total': [],'Patrimônio Líquido Consolidado': []}
                finalbalpassn = pd.DataFrame(finalbalpassn)
        ##Resultados novo
        try:
            excelresn
            finalresn=excelresn
            finalresn = finalresn.fillna(0)
            finalresn = finalresn.T
        except:
            try:
                dfdemresb
                finalresn=dfdemresb
                finalresn = finalresn.fillna(0)
                finalresn = finalresn.T
            except:
                finalresn = {'Resultado Antes dos Tributos sobre o Lucro': []}
                finalresn = pd.DataFrame(finalresn)
        ##Resultados abrangentes novo
        try:
            excelresab
            finalresabn=excelresab
            finalresabn = finalresabn.fillna(0)
            finalresabn = finalresabn.T
        except:
            try:
                dfdemresab
                finalresabn=dfdemresab
                finalresabn = finalresabn.fillna(0)
                finalresabn = finalresabn.T
            except:
                finalresabn = {'': []}
                finalresabn = pd.DataFrame(finalresabn)
        ##Fluxo novo
        try:
            excelflxn
            finalflxn=excelflxn
            finalflxn = finalflxn.fillna(0)
            finalflxn = finalflxn.T
        except:
            try:
                dfflxb
                finalflxn=dfflxb
                finalflxn = finalflxn.fillna(0)
                finalflxn = finalflxn.T
            except:
                finalflxn = {'Saldo Final de Caixa e Equivalentes': []}
                finalflxn = pd.DataFrame(finalflxn)
        ##Valor adicionado novo
        try:
            excelvaladn
            finalvaladn=excelvaladn
            finalvaladn = finalvaladn.fillna(0)
            finalvaladn = finalvaladn.T
        except:
            try:
                dfvaladb
                finalvaladn=dfvaladb
                finalvaladn = finalvaladn.fillna(0)
                finalvaladn = finalvaladn.T
            except:
                finalvaladn = {'': []}
                finalvaladn = pd.DataFrame(finalvaladn)
        #Juntando dados para gráficos
        ##Ativo total
        ativototalconcat = pd.concat([finalbalativo['Ativo Total'],finalbalativon['Ativo Total']])
        ativototalconcat = pd.DataFrame(ativototalconcat)
        ativototalconcat = ativototalconcat.astype(int)
        ##Passivo total
        passivototalconcat = pd.concat([finalbalpassivo['Passivo Total'],finalbalpassn['Passivo Total']])
        passivototalconcat = pd.DataFrame(passivototalconcat)
        passivototalconcat = passivototalconcat.astype(int)
        ##Patrimônio líquido
        finalbalpassnb = finalbalpassn.rename({'Patrimônio Líquido Consolidado': 'Patrimônio Líquido'}, axis=1)
        patliqconcat = pd.concat([finalbalpassivo['Patrimônio Líquido'],finalbalpassnb['Patrimônio Líquido']])
        patliqconcat = pd.DataFrame(patliqconcat)
        patliqconcat = patliqconcat.astype(int)
        ##Lucro Líquido
        lucroconcat = pd.concat([finalflx['Saldo Final de Caixa e Equivalentes'],finalflxn['Saldo Final de Caixa e Equivalentes']])
        lucroconcat = pd.DataFrame(lucroconcat)
        lucroconcat = lucroconcat.astype(int)
        ##EBITDA
        try:
            finaldemres['Resultado Antes Tributaçãoparticipações']
            finaldemresb = finaldemres.rename({'Resultado Antes Tributaçãoparticipações': 'EBITDA'}, axis=1)
        except:
            try:
                finaldemres['Resultado Antes TributaçãoParticipações']
                finaldemresb = finaldemres.rename({'Resultado Antes TributaçãoParticipações': 'EBITDA'}, axis=1)
            except:
                try:
                    finaldemres['Resultado Antes dos Tributos sobre o Lucro']
                    finaldemresb = finaldemres.rename({'Resultado Antes dos Tributos sobre o Lucro': 'EBITDA'}, axis=1)
                except:
                    finaldemresb = finaldemres.rename({'Resultado Antes Tributação/Participações': 'EBITDA'}, axis=1)
        try:
            finalresn['Resultado Antes dos Tributos sobre o Lucro']
            finalresnb = finalresn.rename({'Resultado Antes dos Tributos sobre o Lucro': 'EBITDA'}, axis=1)
        except:
            try:
                finalresn['Resultado Antes Tributaçãoparticipações']
                finalresnb = finalresn.rename({'Resultado Antes Tributaçãoparticipações': 'EBITDA'}, axis=1)
            except:
                try:
                    finalresn['Resultado Antes TributaçãoParticipações']
                    finalresnb = finalresn.rename({'Resultado Antes TributaçãoParticipações': 'EBITDA'}, axis=1)
                except:
                    finalresn['Resultado Antes Tributação/Participações']
                    finalresnb = finalresn.rename({'Resultado Antes Tributação/Participações': 'EBITDA'}, axis=1)
        ebitdacat = pd.concat([finaldemresb['EBITDA'],finalresnb['EBITDA']])
        ebitdacat = pd.DataFrame(ebitdacat)
        ebitdacat = ebitdacat.astype(int)
        #Plotando gráficos
        fig1, f1_axes = plt.subplots(ncols=2, nrows=3, figsize=(25,10))
        fig1.suptitle (ticker, size=50)
        f1_axes[0, 0].plot(ativototalconcat['Ativo Total'], lw=2, marker='.', markersize=10, label="Ativo Total")
        f1_axes[0, 0].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        f1_axes[0, 0].legend()
        f1_axes[0, 1].plot(passivototalconcat['Passivo Total'], lw=2, marker='.', markersize=10, label="Passivo Total")
        f1_axes[0, 1].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        f1_axes[0, 1].legend()
        f1_axes[1, 0].plot(patliqconcat['Patrimônio Líquido'], lw=2, marker='.', markersize=10, label="Patrimônio Líquido")
        f1_axes[1, 0].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        f1_axes[1, 0].legend()
        f1_axes[1, 1].plot(lucroconcat['Saldo Final de Caixa e Equivalentes'], lw=2, marker='.', markersize=10, label="Saldo de Caixa")
        f1_axes[1, 1].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        f1_axes[1, 1].legend()
        f1_axes[2, 0].plot(ebitdacat['EBITDA'], lw=2, marker='.', markersize=10, label="EBITDA")
        f1_axes[2, 0].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        f1_axes[2, 0].legend()
        f1_axes[2, 1].axis('off')
        if not os.path.exists("BVMF/"+setor+"/"+tickerid):
            os.makedirs("BVMF/"+setor+"/"+tickerid)
        plt.savefig("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"data.png")
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"FLUXON/"+tickerid+"_FLUXON.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_FLUXON.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"FLUXON")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO_ANTIGO/"+tickerid+"_BALANCO_ATIVO_ANTIGO.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_BALANCO_ATIVO_ANTIGO.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO_ANTIGO")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO_ANTIGO/"+tickerid+"_BALANCO_PASSIVO_ANTIGO.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_BALANCO_PASSIVO_ANTIGO.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO_ANTIGO")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ANTIGO/"+tickerid+"_RESULTADOS_ANTIGO.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_RESULTADOS_ANTIGO.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ANTIGO")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"FLUXO_ANTIGO/"+tickerid+"_FLUXO_ANTIGO.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_FLUXO_ANTIGO.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"FLUXO_ANTIGO")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO_ANTIGO/"+tickerid+"_VALOR_ADICIONADO_ANTIGO.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_VALOR_ADICIONADO_ANTIGO.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO_ANTIGO")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO/"+tickerid+"_BALANCO_ATIVO.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_BALANCO_ATIVO.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_ATIVO")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO/"+tickerid+"_BALANCO_PASSIVO.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_BALANCO_PASSIVO.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"BALANCO_PASSIVO")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS/"+tickerid+"_RESULTADOS.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_RESULTADOS.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ABRANGENTES/"+tickerid+"_RESULTADOS_ABRANGENTES.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_RESULTADOS_ABRANGENTES.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"RESULTADOS_ABRANGENTES")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"FLUXON/"+tickerid+"_FLUXON.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_FLUXON.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"FLUXON")
        except:
            pass
        try:
            Path("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO/"+tickerid+"_VALOR_ADICIONADO.xlsx").rename("BVMF/"+setor+"/"+tickerid+"/"+tickerid+"_VALOR_ADICIONADO.xlsx")
            os.rmdir("BVMF/"+setor+"/"+tickerid+"/"+"VALOR_ADICIONADO")
        except:
            pass
#Limpando Raw
        if not manterraw == "sim":
            print("Excluindo dados brutos")
            try:
                shutil.rmtree("BVMF/"+setor+"/"+tickerid+"/RAW", ignore_errors=True)
            except:
                pass
#Limpando variáveis
        try:
            finalbalativo
            del finalbalativo
        except:
            pass
        try:
            excelbalativo
            del excelbalativo
        except:
            pass
        try:
            dfbalativoold
            del dfbalativoold
        except:
            pass
        try:
            finalbalpassivo
            del finalbalpassivo
        except:
            pass
        try:
            excelbalpassivo
            del excelbalpassivo
        except:
            pass
        try:
            dfbalpassold
            del dfbalpassold
        except:
            pass
        try:
            finaldemres
            del finaldemres
        except:
            pass
        try:
            exceldemres
            del exceldemres
        except:
            pass
        try:
            dfdemresold
            del dfdemresold
        except:
            pass
        try:
            finalflx
            del finalflx
        except:
            pass
        try:
            excelflx
            del excelflx
        except:
            pass
        try:
            dfflxold
            del dfflxold
        except:
            pass
        try:
            finalvalad
            del finalvalad
        except:
            pass
        try:
            excelvalad
            del excelvalad
        except:
            pass
        try:
            dfvaladold
            del dfvaladold
        except:
            pass
        try:
            finalbalativon
            del finalbalativon
        except:
            pass
        try:
            excelbalativon
            del excelbalativon
        except:
            pass
        try:
            dfbalativob
            del dfbalativob
        except:
            pass
        try:
            finalbalpassn
            del finalbalpassn
        except:
            pass
        try:
            excelbalpassn
            del excelbalpassn
        except:
            pass
        try:
            dfbalpassivob
            del dfbalpassivob
        except:
            pass
        try:
            finalresn
            del finalresn
        except:
            pass
        try:
            excelresn
            del excelresn
        except:
            pass
        try:
            dfdemresb
            del dfdemresb
        except:
            pass
        try:
            finalresabn
            del finalresabn
        except:
            pass
        try:
            excelresab
            del excelresab
        except:
            pass
        try:
            dfdemresab
            del dfdemresab
        except:
            pass
        try:
            finalflxn
            del finalflxn
        except:
            pass
        try:
            excelflxn
            del excelflxn
        except:
            pass
        try:
            dfflxb
            del dfflxb
        except:
            pass
        try:
            finalvaladn
            del finalvaladn
        except:
            pass
        try:
            excelvaladn
            del excelvaladn
        except:
            pass
        try:
            dfvaladb
            del dfvaladb
        except:
            pass
        try:
            dfbalativoraw
            del dfbalativoraw
        except:
            pass
        try:
            dfbalpassivoraw
            del dfbalpassivoraw
        except:
            pass
        try:
            dfdemresraw
            del dfdemresraw
        except:
            pass
        try:
            dfdemresaraw
            del dfdemresaraw
        except:
            pass
        try:
            dfflxraw
            del dfflxraw
        except:
            pass
        try:
            dfvaladraw
            del dfvaladraw
        except:
            pass
#Fechamento do if de código CVM
    else:
        print("Código CVM inválido: "+i+".")
        driver.quit()
#Fechamento geral
driver.quit()
print("Finalizado.")

Nome do arquivo com os códigos na primeira coluna:  CÓDIGOS_CVM_ativos
Datas separadas por vírgula (Deixe vazio para todas):  
Digite "SIM" para manter arquivos brutos:  SIM


Dados brutos serão salvos
Código CVM válido: 3 ().
Setor não identificado
Datas: 2016,2017,2018,2019,2020,2021.
 completo.
Código CVM inválido: 35.
Código CVM válido: 94 (PANATLANTICA S.A.).
MATERIAIS_BÁSICOS/SIDERURGIA_E_METALURGIA/ARTEFATOS_DE_FERRO_E_AÇO
Datas: 1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021.
Formulário de PANATLANTICA S.A. em 1997 no modelo BVMF antigo.
Balanço Ativo de PANATLANTICA S.A. em 1997 sem dados.
Balanço Passivo de PANATLANTICA S.A. em 1997 sem dados.
Fluxo de Caixa de PANATLANTICA S.A. em 1997 sem dados.
Valor adicionado de PANATLANTICA S.A. em 1997 sem dados.
Formulário de PANATLANTICA S.A. em 1998 no modelo BVMF antigo.
Balanço Ativo de PANATLANTICA S.A. em 1998 sem dados.
Balanço Passivo de PANATLANTICA S.A. em 1998 sem dados.
Fluxo de Caixa de PANATLANTICA S.A. em 1998 sem dados.
Valor adicionado de PANATLANTICA S.A. em 1998 sem dados.
Formulário de PANATLANTICA S.A. em 1999